In [26]:
import pandas as pd
import numpy as np
import calmap
import os
import re

In [27]:
PATH='C:/Users/harsh/Documents/cricket-analysis/cricket-data-processing/datasets/'
def load_match_data(filenames):
    match_regex = re.compile(r'^(?:.*)_match.csv$')
    matches = [m for m in map(match_regex.match, filenames) if m is not None]
    for match in matches:
        match_type = re.sub('\_match.csv$', '', match.group(0))
        print("Match:"+match_type)
        dataset = pd.read_csv(PATH+match.group(0))
        dataset.insert(2, "type", match_type)
        yield dataset

def load_innings_data(filenames):
    inning_regex = re.compile(r'^(?:.*)_innings.csv$')
    innings = [m for m in map(inning_regex.match, filenames) if m is not None]
    for inning in innings:
        inning_type = re.sub('\_innings.csv$', '', inning.group(0))
        print("Inning:"+inning_type)
        dataset = pd.read_csv(PATH+inning.group(0))
        dataset.insert(2, "type", inning_type)
        yield dataset

data_match=pd.concat(load_match_data(os.listdir(PATH)))
data_innings=pd.concat(load_innings_data(os.listdir(PATH)))
print(data_match.size)
print(data_innings.size)

Match:odi
Match:t20
Match:test
Inning:odi


C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:23: DtypeWarning: Columns (1,2,3,6,7,8,9,17,18,20,21,22,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:23: DtypeWarning: Columns (1,2,3,6,7,8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.


Inning:t20
Inning:test
177096
11702428


In [28]:
df = pd.merge(
  data_innings,
  data_match,
  on=['Date','Ground']
)
df.head()

df=df[df['Match Year']==2018]

In [29]:
def distinctValue(dataset, key):
    return dataset[key].unique()

In [30]:
# For winner
# df.to_csv('df.csv')
# print(df.iloc[:88,:])
# first_match = df.iloc[:88,:]
# # print(len(df['Date'].unique()))
# countries = distinctValue(first_match,'Country_y')
# for country in countries:
#     for row in first_match.head().itertuples():
# #         print(row.Country_y)
#         if (country == row.Country_y):
#             print(row.Country_y, row.Result)
# #             print(row.Result)
# result = distinctValue(first_match,'Result')
# match_date = distinctValue(first_match, 'Date')
# # print(first_match[['Innings Player','Innings Runs Scored']])
# # first_match = first_match.sort_values(by=['Country_y']).unique(['Country_y'])
# print(match, result, match_date)
# first_match.to_csv('first_match.csv')
# print(df)
# print(df[Date])
df

,Innings Player,Innings Runs Scored,type_x,Innings Runs Scored Num,Innings Minutes Batted,Innings Batted Flag,Innings Not Out Flag,Innings Balls Faced,Innings Boundary Fours,Innings Boundary Sixes,...,Result,Margin,type_y,Match,Home/Away,Match Month,Match Year,Match Period,Matches,Country_y
0,JJ Roy,180,odi,180,151,1.0,0.0,151,16,5,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
1,JJ Roy,180,odi,180,151,1.0,0.0,151,16,5,...,Lost,5 wickets,odi,Australia v England,Home,Jan,2018,Jan-18,1,Australia
2,JE Root,91*,odi,91,186,1.0,1.0,110,5,0,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
3,JE Root,91*,odi,91,186,1.0,1.0,110,5,0,...,Lost,5 wickets,odi,Australia v England,Home,Jan,2018,Jan-18,1,Australia
4,JM Bairstow,14,odi,14,22,1.0,0.0,11,2,0,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812698,Imrul Kayes,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,151 runs,test,Bangladesh v Zimbabwe,Home,Nov,2018,Nov-18,1,Bangladesh
812699,Liton Das,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Won,151 runs,test,Bangladesh v Zimbabwe,Away,Nov,2018,Nov-18,1,Zimbabwe
812700,Liton Das,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,151 runs,test,Bangladesh v Zimbabwe,Home,Nov,2018,Nov-18,1,Bangladesh
812701,Najmul Hossain Shanto,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Won,151 runs,test,Bangladesh v Zimbabwe,Away,Nov,2018,Nov-18,1,Zimbabwe


In [31]:
# For Innings
def innings(first_match):
    # first_match = df.iloc[:88,:]
    # inning_id
    # score_id
    # match_type
    # team1
    # team2
    # run
    # wickets
    # playing11
    try:
        teams = first_match.Country_y.unique()
        team=["",""]
        team_runs=[0,0]
        team_wicket=[0,0]
        match_type = first_match.type_x.unique()[0]

        # Runs Conceded
        first_match["Innings Runs Conceded"]=first_match["Innings Runs Conceded"].fillna(value=0)
        first_match.loc[first_match["Innings Runs Conceded"]=="-","Innings Runs Conceded"]=0
        first_match["Innings Runs Conceded"]=first_match["Innings Runs Conceded"].astype(int)
        for count,(key,value) in enumerate(first_match.groupby("Country_x")["Innings Runs Conceded"].agg("sum").iteritems(),0):
            team[count]=key
            team_runs[1-count]=value/2
        team1 = team[0]
        team2 = team[1]
        team1_run = team_runs[0]
        team2_run = team_runs[1]

        # Wickets Taken
        first_match["Innings Wickets Taken"]=first_match["Innings Wickets Taken"].fillna(value=0)
        first_match.loc[first_match["Innings Wickets Taken"]=="-","Innings Wickets Taken"]=0
        first_match["Innings Wickets Taken"]=first_match["Innings Wickets Taken"].astype(int)
        for count,(key,value) in enumerate(first_match.groupby("Country_x")["Innings Wickets Taken"].agg("sum").iteritems(),0):
            team_wicket[1-count]=value/2

        team1_wicket = team_wicket[0]
        team2_wicket = team_wicket[1]

        # Playing11
        team1_p11 = ",".join(first_match[first_match.Country_x == team1]['Innings Player'].unique())
        team2_p11 = ",".join(first_match[first_match.Country_x == team2]['Innings Player'].unique())

        innings = {
            'match_type': match_type,
            'team1_name': team1,
            'team2_name': team2,
            'team1_run': team1_run,
            'team2_run': team2_run,
            'team1_wicket': team1_wicket,
            'team2_wicket': team2_wicket,
            'team1_p11': team1_p11,
            'team2_p11': team2_p11,
            'Date': first_match.Date.unique()
        }

        df1 = pd.DataFrame (innings, columns = ['Date','match_type','team1_name','team2_name','team1_run','team2_run','team1_wicket'
                                               ,'team2_wicket','team1_p11', 'team2_p11'],index=[0])
        return df1

    except Exception as e:
        print('Skipped for--- ', e, first_match.Date.unique())
#         pass

    # print (df1)

    # df1.to_csv('temp.csv')

    # print(match_type, team1, team2, team1_run, team2_run,team1_wicket, team2_wicket, team1_p11, team2_p11)

In [32]:
# ScoreCard
# team1
# team2
# winning_team
# loosing_team
# Date
# win_matgin

def scoreCard(match):
#     print(match.Country_y.unique())
#     match = df.iloc[:88,:]
    try:
        teams = match.Country_y.unique()
        team1 = teams[0]
        team2 = teams[1]
        winning_team = match[match.Result == 'Won']['Country_y'].unique()
        loosing_team = match[match.Result == 'Lost']['Country_y'].unique()
        Date = match.Date.unique()
        win_margin = match[match.Result == 'Won']['Margin'].unique()

        score_card = {
                        'team1_name': team1,
                        'team2_name': team2,
                        'winning_team': winning_team,
                        'loosing_team': loosing_team,
                        'Date': Date,
                        'win_margin': win_margin
                    }

        df1 = pd.DataFrame (score_card, columns = ['Date','team1_name','team2_name',
                                                            'winning_team','loosing_team','win_margin'],index=[0])
        return df1
    except:
        print('skipped')
        pass
# print(df1)
# print(team1, team2, winning_team, loosing_team, Date, win_margin)


In [33]:
# Player performance for perticular match
# Player Name
# Match
# Date
# Runs
# Wickets
# strike_rate
# economy_rate
# minutes_played

match = df.iloc[:88,:]
# player_name = []
player_score = []
# match = match.Match.unique()
teams = match.Country_y.unique()
# Date = match.Date.unique()
print(teams)
# Runs Conceded
match["Innings Runs Scored Num"]= match["Innings Runs Scored Num"].fillna(value=0)
match.loc[match["Innings Runs Scored Num"]=="-","Innings Runs Scored Num"]=0
match["Innings Runs Scored Num"]=match["Innings Runs Scored Num"].astype(int)
for count,(key,value) in enumerate(match.groupby("Innings Player")["Innings Runs Scored Num"].agg("sum").iteritems(),0):
    player_score.append(key)
    player_score.append(value/2)
player_score = ','.join(map(str, player_score))
# team1 = team[0]
# team2 = team[1]
# team1_run = team_runs[0]
# team2_run = team_runs[1]
df

['England' 'Australia']


C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

,Innings Player,Innings Runs Scored,type_x,Innings Runs Scored Num,Innings Minutes Batted,Innings Batted Flag,Innings Not Out Flag,Innings Balls Faced,Innings Boundary Fours,Innings Boundary Sixes,...,Result,Margin,type_y,Match,Home/Away,Match Month,Match Year,Match Period,Matches,Country_y
0,JJ Roy,180,odi,180,151,1.0,0.0,151,16,5,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
1,JJ Roy,180,odi,180,151,1.0,0.0,151,16,5,...,Lost,5 wickets,odi,Australia v England,Home,Jan,2018,Jan-18,1,Australia
2,JE Root,91*,odi,91,186,1.0,1.0,110,5,0,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
3,JE Root,91*,odi,91,186,1.0,1.0,110,5,0,...,Lost,5 wickets,odi,Australia v England,Home,Jan,2018,Jan-18,1,Australia
4,JM Bairstow,14,odi,14,22,1.0,0.0,11,2,0,...,Won,5 wickets,odi,Australia v England,Away,Jan,2018,Jan-18,1,England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812698,Imrul Kayes,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,151 runs,test,Bangladesh v Zimbabwe,Home,Nov,2018,Nov-18,1,Bangladesh
812699,Liton Das,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Won,151 runs,test,Bangladesh v Zimbabwe,Away,Nov,2018,Nov-18,1,Zimbabwe
812700,Liton Das,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,151 runs,test,Bangladesh v Zimbabwe,Home,Nov,2018,Nov-18,1,Bangladesh
812701,Najmul Hossain Shanto,NaN,test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Won,151 runs,test,Bangladesh v Zimbabwe,Away,Nov,2018,Nov-18,1,Zimbabwe


In [34]:
df2 = pd.DataFrame()
index = 0
for date in df.Date.unique():
    dfs=df[df.Date==date]
    
#     -----innings call-----
#     single_inning = innings(dfs)
#     df2 = df2.append(innings(dfs), ignore_index = True)

#     -----score_card call-----
#     single_inning = score_card(dfs)
    df2 = df2.append(scoreCard(dfs), ignore_index = True)

#     if (index == 100):
#         print('breaking')
#         break
#     index = index +1

df2.to_csv('score_card.csv')
print('doneee')

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
doneee
